# Graphing cone calorimeter data
Note: for best results, view this notebook in VSCode with the Jupyter extension installed, not JupyterLab.

In [12]:
# install dependencies
# %pip install pandas
# %pip install plotly
# %pip install scipy
# %pip install ipywidgets

In [13]:
import pandas as pd
import scipy
import numpy as np

import json
from pathlib import Path
from datetime import date

import ipywidgets as widgets
from IPython.display import display, clear_output

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils import colorize

## Same file

In [ ]:
# some basic setup
out = widgets.Output()

input_path = Path("../OUTPUT/")

folders = [Path(*x.parts[2:]) for x in list(input_path.rglob("./*/"))]

# filter out folders that don't contain any metadata files (i.e. have no .json files in them)
folders = [x for x in folders if len(list(Path(input_path / Path(x)).glob("*.json"))) > 0]

source_widget = widgets.Dropdown(options=folders, description="Source:")

metadata_files = list(Path(input_path / Path(source_widget.value)).glob("*.json"))

filename_widget = widgets.Dropdown(options=[str(x.stem) for x in metadata_files], description="Filename:")

data = pd.read_csv(input_path / Path(source_widget.value) / Path(filename_widget.value + ".csv"))

# UI code

def update_folder(event=None):
    # get test metadata
    global metadata_files
    metadata_files = list(Path(input_path / Path(source_widget.value)).glob("*.json"))

    global filename_widget
    if len(metadata_files) > 0:
        filename_widget = widgets.Dropdown(options=[str(x.stem).replace("_metadata", "") for x in metadata_files], description="Filename:")
        plot()
    else:
        filename_widget = widgets.Dropdown(options=["-"], description="Filename:")
        redraw_ui()
        

# Select X & Y axis
x_axis_widget = widgets.Dropdown(description='X axis:', options=data.columns)
y_axis_widget = widgets.Dropdown(description='Y axis:', options=data.columns)
sec_y_axis_widget = widgets.Dropdown(description='Sec. Y axis:', options=data.columns)
sec_x_axis_widget = widgets.Dropdown(description='Sec. X axis:', options=data.columns)

y_max = widgets.FloatText(description='Y max:', value=0)
y_min = widgets.FloatText(description='Y min:', value=0)
y_scale_enabled = widgets.Checkbox(description='Y scale:', value=False)
savgol_filter_enabled = widgets.Checkbox(description='Savgol filter', value=False)
derivative_enabled = widgets.Checkbox(description='Derivative', value=False)

sec_y_max = widgets.FloatText(description='Sec. Y max:', value=0)
sec_y_min = widgets.FloatText(description='Sec. Y min:', value=0)
sec_y_scale_enabled = widgets.Checkbox(description='Sec. Y scale:', value=False)
sec_savgol_filter_enabled = widgets.Checkbox(description='Savgol filter', value=False)
sec_derivative_enabled = widgets.Checkbox(description='Derivative', value=False)

update = widgets.Button(description="Plot", icon="check")

fig = make_subplots(specs=[[{"secondary_y": True}]])
    
def redraw_ui():
    clear_output()

    primary_box = widgets.Box(children=[x_axis_widget, y_axis_widget, y_scale_enabled, y_min, y_max, savgol_filter_enabled, derivative_enabled])
    secondary_box = widgets.Box(children=[sec_x_axis_widget, sec_y_axis_widget, sec_y_scale_enabled, sec_y_min, sec_y_max, sec_savgol_filter_enabled, sec_derivative_enabled])
    vbox = widgets.VBox([source_widget, filename_widget, primary_box, secondary_box, update])

    display(vbox)

def plot(event=None):

    data = pd.read_csv(input_path / Path(source_widget.value) / Path(filename_widget.value + ".csv"))
    
    x_axis_widget.options = data.columns
    y_axis_widget.options = data.columns

    sec_y_axis_widget.options = data.columns
    sec_x_axis_widget.options = data.columns

    redraw_ui()

    y_data = data[y_axis_widget.value]
    x_data = data[x_axis_widget.value]

    sec_y_data = data[sec_y_axis_widget.value]
    sec_x_data = data[sec_x_axis_widget.value]

    # Optionally add savgol filter to the data
    if savgol_filter_enabled.value:
        y_data = scipy.signal.savgol_filter(y_data, 31, 3)
    if sec_savgol_filter_enabled.value:
        sec_y_data = scipy.signal.savgol_filter(sec_y_data, 31, 3)

    if derivative_enabled.value:
        y_data = np.gradient(y_data, x_data)
    if sec_derivative_enabled.value:
        sec_y_data = np.gradient(sec_y_data, x_data)



    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if y_axis_widget.value != x_axis_widget.value:
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines', name=f"{y_axis_widget.value} vs {x_axis_widget.value}"), secondary_y=False)
        if y_scale_enabled.value:
            fig.update_yaxes(range=[y_min.value, y_max.value], autorange=False)
    
    if sec_y_axis_widget.value != sec_x_axis_widget.value:
        fig.add_trace(go.Scatter(x=sec_x_data, y=sec_y_data, mode='lines', name=f"{sec_y_axis_widget.value} vs {sec_x_axis_widget.value} (secondary)"), secondary_y=True)
        if sec_y_scale_enabled.value:
            fig.update_yaxes(range=[sec_y_min.value, sec_y_max.value], secondary_y=True, autorange=False)

    fig.show()

# TODO: also allow setting the scale for each axis

filename_widget.observe(plot, names="value")
source_widget.observe(update_folder, names="value")

update.on_click(plot)

plot()


- Check Pe data - see which tests are actually useful by plotting HRR
    - if Pe data is not meaningful & HRR doesn't seem to be useful, remove test
- do the same for I_o and I, but don't remove tests, just don't calculate K_smoke
- for each of these, plot multiple at a time on the same graph

- Create spreadsheet with metadata for all tests

# Findings
- 928 tests have negative Pe data (at least one row)
- 1547 tests have bad light intensity (I_o or I) data (at least one row)
    - i.e. either missing values or negative values

In [ ]:
# print out material info again:
metadata_files = list(Path(input_path / Path(source_widget.value)).glob("**/*.json"))

for metadata_file in metadata_files:
    test_name = metadata_file.stem.replace("_metadata", "")
    metadata = json.load(metadata_file.open())
    print(colorize(f"Test: {test_name}", "blue"))
    print(f" - Date: {metadata.get('date')}")
    print(f" - Heat flux (kw/m^2): {metadata.get('heat_flux_kw/m^2')}")
    print(f" - Material: {metadata.get('material_name')}")
    print(f" - Specimen description: {metadata.get("specimen_description")}")
    print(f" - General comments: {metadata.get('comments')}")

Test: 17110002
 - Date: 2017-11-06T15:12:00
 - Heat flux (kw/m^2): 50.0
 - Material: blank/001
 - Specimen description: Radiator Data -002
 - General comments: None
Test: 18020001-FTT PMMA
 - Date: 2018-02-13T13:30:00
 - Heat flux (kw/m^2): 50.0
 - Material: PMMA
 - Specimen description: PMMA
 - General comments: None
Test: 18020002
 - Date: 2018-02-13T14:25:00
 - Heat flux (kw/m^2): 50.0
 - Material: uncoated wood
 - Specimen description: uncoated wood
 - General comments: None
Test: 18040001
 - Date: 2018-04-17T15:31:00
 - Heat flux (kw/m^2): 50.0
 - Material: PMMA eplastics
 - Specimen description: PMMA 04172018
 - General comments: None
Test: 18040010
 - Date: 2018-04-20T14:09:00
 - Heat flux (kw/m^2): 50.0
 - Material: Coating A1 1st series 60 mm
 - Specimen description: Coating A1 1st series 60 mm
 - General comments: None
Test: 18050001
 - Date: 2018-05-17T10:16:00
 - Heat flux (kw/m^2): 75.0
 - Material: None
 - Specimen description: Data for Tony
 - General comments: None
Test